# Create A Warped Disk Setup for RADMC-3D

## Imports

In [ ]:
import os
from pathlib import Path
import subprocess

from matplotlib import cm
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
from tqdm.auto import tqdm
from IPython.display import Video
from astropy import constants as c

from radmc3dPy import analyze, natconst
from radmc3dPy.image import readImage, plotImage, makeImage

au = c.au.cgs.value
pc = c.pc.cgs.value
M_sun = c.M_sun.cgs.value
L_sun = c.L_sun.cgs.value
R_sun = c.R_sun.cgs.value
Grav = c.G.cgs.value
m_p = c.m_p.cgs.value

In [ ]:
%matplotlib widget

In [ ]:
from diskwarp.helper import warp_transformation
from diskwarp.helper import unwarp_transformation
from diskwarp.helper import logistic
from diskwarp.helper import vel_car_to_sph
from diskwarp.helper import vel_sph_to_car
from diskwarp.helper import grid_refine_inner_edge
from diskwarp.helper import call_radmc
from diskwarp import helper

## Setup 

### Set general parameters

In [ ]:
# Monte Carlo parameters
#
nphot_therm = 1000000
nphot_scat = 1000000

In [ ]:
# Grid parameters
#
n_r = 200
n_theta = 128
n_phi = 256
r_in = 0.5 * au
r_out = 100 * au
thetaup = 0.1       # Theta grid starting point (0=pole, but singular, so choose >0)

In [ ]:
# inner edge grid refinement
nlev_rin = 8
nspan_rin = 3

In [ ]:
# Disk parameters
#
sigma_0 = 1e1           # Gas surface density at 1 au [g / cm^2]
d2g = 0.01              # dust-to-gas ratio
gamma_gas = 1.0         # power-law exponent of the surface density
hor_0 = 0.05            # h/r at 1 au
hor_exp = 0.1           # flaring exponent

In [ ]:
# Star parameters
#
M_star = 2.4 * M_sun
R_star = 2.4 * R_sun
T_star = 1e4
star_coord = np.array([0., 0., 0.])

### Make the coordinates

This creates the grid that gets put into RADMC3D

In [ ]:
ri = np.geomspace(r_in, r_out, n_r + 1)
ri = grid_refine_inner_edge(ri, nlev_rin, nspan_rin)   # Refinement at inner edge
thetai = np.linspace(thetaup, np.pi - thetaup, n_theta + 1)
phii = np.linspace(0.0, 2.0 * np.pi, n_phi + 1)
rc = 0.5 * (ri[:-1] + ri[1:])
thetac = 0.5 * (thetai[:-1] + thetai[1:])
phic = 0.5 * (phii[:-1] + phii[1:])
n_r = len(rc)     # Recompute nr, because of refinement at inner edge

In [ ]:
# Make the full mesh
RC, THETAC, PHIC = np.meshgrid(rc, thetac, phic, indexing='ij')
XC = RC * np.sin(THETAC) * np.cos(PHIC)
YC = RC * np.sin(THETAC) * np.sin(PHIC)
ZC = RC * np.cos(THETAC)

### Make the warped model

Here we compute the original position of every grid cell before the disk was warped

In [ ]:
# un-warp the coordinates
warp = 45.
twist = 0.

warp_array = helper.warp(rc, i_in=warp, r0=75 * au, dr=10 * au)  # Specify the r0 and dr in AU
twist_array = helper.twist(rc, phi=twist, r0=75 * au, dr=10 * au)

XU, YU, ZU = unwarp_transformation(
    XC, YC, ZC,
    warp_array[:, None, None],
    twist_array[:, None, None])

RU = np.sqrt(XU**2 + YU**2 + ZU**2)
THETAU = np.pi / 2.0 - np.arctan2(ZU, np.sqrt(XU**2 + YU**2))
PHIU = (np.arctan2(YU, XU) + 2 * np.pi)%(2 * np.pi)

For those positions, we compute the gas and dust denstiy.

In [ ]:
# Make the dust density model
sig_g = sigma_0 * (RU / au)**-gamma_gas
H = hor_0 * (rc / au)**hor_exp * rc
rho_g = (sig_g / (np.sqrt(2. * np.pi) * H[:, None, None])) * np.exp(-(ZU**2 / H[:, None, None]**2) / 2.0)
rho_d = d2g * rho_g

In [ ]:
# determine where to put the CO and compute CO abundance
em_surf = (np.abs(ZU) > 4.0 * H[:, None, None]) & (np.abs(ZU) < 5.0 * H[:, None, None])

CO_abundance = 1e-4
fact_CO = CO_abundance / (2.3 * m_p)
nco = rho_g * fact_CO * em_surf

In [ ]:
# Make the gas temperature model
T_0 = 300.
q = 0.5
tgas = T_0 * (RU / (50.0 * au))**(-q)

In [ ]:
# Make the velocity model
# this velocity is the original r,theta,phi velocity in the un-warped state,
# before warping to our coordinates
# wasn't yet really computed
VRU = np.zeros_like(RU)
VTU = np.zeros_like(RU)
VPU = np.sqrt(Grav * M_star / RU)

# make cartesian velocities

VXU, VYU, VZU = vel_sph_to_car(THETAU, PHIU, VRU, VTU, VPU)

# warp velocities
VXW, VYW, VZW = warp_transformation(
    VXU, VYU, VZU,
    warp_array[:, None, None],
    twist_array[:, None, None])

VRW, VTW, VPW = vel_car_to_sph(THETAC, PHIC, VXW, VYW, VZW)

# set the turbulent velocity contribution

vturb = 0.001 * VPU

In [ ]:
it = int(n_theta // 2)
s = 5
_r = 40

f, ax = plt.subplots(subplot_kw={'projection':'3d'}, figsize=(10, 10))
ax.set_proj_type('ortho')

ir = rc.searchsorted(_r * au)
ax.set_xlim(-_r, _r)
ax.set_ylim(-_r, _r)
ax.set_zlim(-_r, _r)

ax.view_init(elev=5, azim=120)

# plot the azimuthal ring
ax.plot3D(XC[ir, it, :] / au, YC[ir, it, :] / au, ZC[ir, it, :] / au, 'k', lw=0.5)
ax.plot3D(XU[ir, it, :] / au, YU[ir, it, :] / au, ZU[ir, it, :] / au, 'r', lw=0.5)

#scale = 5000000
scale = .3 * _r * au / (np.sqrt(VXW[ir, it, :]**2 + VYW[ir, it, :]**2 + VZW[ir, it, :]**2)).mean()

arr_u = np.array([[XU, XU + scale * VXU], [YU, YU + scale * VYU], [ZU, ZU + scale * VZU]])
arr_w = np.array([[XC, XC + scale * VXW], [YC, YC + scale * VYW], [ZC, ZC + scale * VZW]])

for iphi in range(0, n_phi, s):
    ax.plot3D(*arr_u[:, :, ir, it, iphi]/au, 'r', lw=0.5)
    ax.plot3D(*arr_w[:, :, ir, it, iphi]/au, 'k', lw=0.5)
    
ax.view_init(0, 0)

### Write RADMC3D setup

In [ ]:
# Write the wavelength_micron.inp file
#
lam1 = 0.1e0
lam2 = 7.0e0
lam3 = 25.e0
lam4 = 1.0e4
n12 = 20
n23 = 100
n34 = 30
lam12 = np.geomspace(lam1, lam2, n12, endpoint=False)
lam23 = np.geomspace(lam2, lam3, n23, endpoint=False)
lam34 = np.geomspace(lam3, lam4, n34, endpoint=True)
lam = np.concatenate([lam12, lam23, lam34])
nlam = lam.size

In [ ]:
# Write the wavelength file
#
with open('wavelength_micron.inp', 'w+') as f:
    f.write('%d\n' % (nlam))
    for value in lam:
        f.write('%13.6e\n' % (value))

In [ ]:
# Write the stars.inp file
#
with open('stars.inp', 'w+') as f:
    f.write('2\n')
    f.write('1 %d\n\n' % (nlam))
    f.write('%13.6e %13.6e %13.6e %13.6e %13.6e\n\n' % (R_star, M_star, *star_coord))
    for value in lam:
        f.write('%13.6e\n' % (value))
    f.write('\n%13.6e\n' % (-T_star))

In [ ]:
# Write the grid file
#
with open('amr_grid.inp', 'w+') as f:
    f.write('1\n')                       # iformat
    f.write('0\n')                       # AMR grid style  (0=regular grid, no AMR)
    f.write('100\n')                     # Coordinate system: spherical
    f.write('0\n')                       # gridinfo
    f.write('1 1 1\n')                   # Include r,theta coordinates
    f.write('%d %d %d\n' % (n_r, n_theta, n_phi))  # Size of grid
    for value in ri:
        f.write('%13.6e\n' % (value))      # X coordinates (cell walls)
    for value in thetai:
        f.write('%13.6e\n' % (value))      # Y coordinates (cell walls)
    for value in phii:
        f.write('%13.6e\n' % (value))      # Z coordinates (cell walls)

In [ ]:
# Write the density file
#
with open('dust_density.inp','w+') as f:
    f.write('1\n')                       # Format number
    f.write('%d\n'%(n_r * n_theta * n_phi))     # Nr of cells
    f.write('1\n')                       # Nr of dust species
    data = rho_d.ravel(order='F')   # Create a 1-D view, fortran-style indexing
    data.tofile(f, sep='\n', format="%13.6e")
    f.write('\n')

In [ ]:
# Dust opacity control file
#
with open('dustopac.inp', 'w+') as f:
    f.write('2               Format number of this file\n')
    f.write('1               Nr of dust species\n')
    f.write('============================================================================\n')
    f.write('1               Way in which this dust species is read\n')
    f.write('0               0=Thermal grain\n')
    f.write('silicate        Extension of name of dustkappa_***.inp file\n')
    f.write('----------------------------------------------------------------------------\n')

In [ ]:
# write CO density

with open('numberdens_co.inp', 'w+') as f:
    f.write('1\n')                       # Format number
    f.write('%d\n' % (n_r * n_theta * n_phi))     # Nr of cells
    data = nco.ravel(order='F')          # Create a 1-D view, fortran-style indexing
    data.tofile(f, sep='\n', format="%13.6e")
    f.write('\n')

In [ ]:
# Write the gas velocity field
#
with open('gas_velocity.inp', 'w+') as f:
    f.write('1\n')                       # Format number
    f.write('%d\n' % (n_r * n_theta * n_phi))     # Nr of cells
    for iphi in range(n_phi):
        for itheta in range(n_theta):
            for ir in range(n_r):
                f.write('%13.6e %13.6e %13.6e\n' % (VRW[ir, itheta, iphi], VTW[ir, itheta, iphi], VPW[ir, itheta, iphi]))

In [ ]:
# Write the microturbulence file
#
with open('microturbulence.inp', 'w+') as f:
    f.write('1\n')                       # Format number
    f.write('%d\n' % (n_r * n_theta * n_phi))     # Nr of cells
    data = vturb.ravel(order='F')        # Create a 1-D view, fortran-style indexing
    data.tofile(f, sep='\n', format="%13.6e")
    f.write('\n')

In [ ]:
# Write the lines.inp control file
#
with open('lines.inp', 'w') as f:
    f.write('1\n')
    f.write('1\n')
    f.write('co    leiden    0    0\n')

In [ ]:
# Write the radmc3d.inp control file
#
with open('radmc3d.inp', 'w+') as f:
    f.write('nphot = %d\n' % (nphot_therm))
    f.write('nphot_scat = %d\n' % (nphot_scat))
    f.write('scattering_mode_max = 1\n')
    f.write('iranfreqmode = 1\n')
    f.write('tgas_eq_tdust = 1\n')

### Check density contours of dust and gas

In [ ]:
data = analyze.readData(ddens=True, gdens=True, ispec='co', binary=False)

In [ ]:
fig, ax = plt.subplots()
c = ax.pcolormesh(data.grid.x/natconst.au, np.pi/2.-data.grid.y, data.ndens_mol[:,:,20,0].T, norm=LogNorm(vmin=1e-20, vmax=1e-10))
ax.set_xlabel('r [AU]')
ax.set_ylabel(r'$\pi/2-\theta$')
# ax.set_xscale('log')

cb = fig.colorbar(c)
cb.set_label(r'$\log_{10}{\rho}$', rotation=270.)

Make a movie of this $\phi$-sweep

In [ ]:
dirname = Path('frames')
dirname.mkdir(exist_ok=True)

for iphi,_phi in tqdm(enumerate(phic), total=len(phic)):
    # c.set_array(data.rhodust[:,:,iphi,0].T.ravel())
    c.set_array(data.ndens_mol[:,:,iphi,0].T.ravel())
    ax.set_title(f'$\phi = {np.rad2deg(_phi):.2f}$º')
    fig.savefig(dirname / f'frame_{iphi:03d}.jpeg')
    
p = subprocess.getoutput(f'ffmpeg -y -framerate 30 -i {dirname}/frame_%03d.jpeg -c:v libx264 -crf 23 -pix_fmt yuv420p video.mp4')

Video('video.mp4', width=500, height=500, html_attributes='loop controls autoplay') 

## Run RADMC-3D

General image parameters

In [ ]:
dpc         = 140.     # Distance in parsec (for conversion to Jy/pixel in 1.3 mm map)
incl        = -30.
phi         = 0.
npix        = 200
sizeau      = 200
lamda_image = 1.3004e3

### Thermal Monte Carlo

In [ ]:
rc, output = call_radmc('radmc3d mctherm setthreads 8', total=nphot_therm)

### Continuum image

In [ ]:
rc, output = call_radmc(f'radmc3d image dpc {dpc} incl {incl} phi {phi} lambda {lamda_image} sizeau {sizeau} npix {npix} setthreads 8', total=nphot_scat)

In [ ]:
im_mm = readImage()
f, ax = plt.subplots(figsize=(10, 8))
plotImage(im_mm, au=True, log=True, maxlog=3, bunit='inu', dpc=dpc, cmap='magma', ax=ax);

### Scattered light image

In [ ]:
rc, output = call_radmc(f'radmc3d image dpc {dpc} incl {incl} phi {phi} lambda 1.65 sizeau {sizeau} npix {npix} setthreads 8', total=nphot_scat)

In [ ]:
im_mm = readImage()
f, ax = plt.subplots(figsize=(10, 8))
plotImage(im_mm, au=True, log=True, maxlog=7, bunit='inu', dpc=dpc, cmap='magma', ax=ax);

### Single Channel Map

In [ ]:
# Now let's make a set of channel maps
vkms = -1
rc, output = call_radmc(f'radmc3d image imolspec 1 iline 2 vkms {vkms} incl {incl} phi {phi} npix {npix} setthreads 8 sizeau {sizeau}', total=nphot_scat)

Read and plot the image

In [ ]:
im = readImage()
f, ax = plt.subplots(figsize=(8, 6))
plotImage(im, au=True, cmap=cm.hot, ax=ax, maxlog=3, log=True)

### Try to make momentmaps

In [ ]:
# Create channelmaps in a range of +- 5km/s'
widthkms = 5.
linenlam = 40
#os.system(f'radmc3d image iline 2 widthkms {widthkms} linenlam {linenlam} incl {incl} phi {phi} npix {npix} setthreads 4 sizeau {sizeau} nphot_spec {nphot_scat}')
rc, output = call_radmc(f'radmc3d image iline 2 widthkms {widthkms} linenlam {linenlam} incl {incl} phi {phi} npix {npix} setthreads 8 sizeau {sizeau} nphot_spec {nphot_scat}', total=linenlam * nphot_scat)

In [ ]:
im = readImage()

# Write fits file
fname=str(f'cube_incl{incl}phi{phi}_inc{twist}_PA{warp}.fits')
nu0 = 230.538*10**9 # Hz
im.writeFits(fname=fname, dpc=dpc, coord='03h10m05s -10d05m30s',
             bandwidthmhz=2000.0, casa=False, nu0=nu0, stokes='I')

In [ ]:
f, ax = plt.subplots()
ax.plot(im.freq / 1e9, im.image[25, 100, :], '-x')

In [ ]:
moment = 1
wav0 = lamda_image
vrange = [-1.0, 1.0]
f, ax = plt.subplots()
im.plotMomentMap(moment=1, wav0=wav0, dpc=dpc, au=True, cmap='RdBu', vclip=vrange)
f.savefig('moment1.pdf', transparent=True, bbox_inches='tight')

In [ ]:
import bettermoments as bm
data, velax = bm.load_cube(fname)
rms = bm.estimate_RMS(data=data, N=5)
moments = bm.collapse_first(velax=velax, data=data, rms=rms)
bm.save_to_FITS(moments=moments, method='first', path=fname)